In [ ]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
#Inidividual Title Search
query = input('What title are you looking for?')

In [ ]:
#Set the URL
base_url = 'https://reelgood.com'
url = f"{base_url}/search?q='{query}'"
print(url)

In [ ]:
# Retrieve page with the requests
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
link = soup.select('.e1qyeclq5 a')[0]['href']
show_url = base_url + link
print(show_url)

In [ ]:
#Prepping response
show_response = requests.get(show_url)
show_soup = BeautifulSoup(show_response.text, 'html.parser')

In [ ]:
#Narrowing Script
services = show_soup.select('.e126mwsw1 span[class*=hou113]')

In [ ]:
#Iterative Scrape
title = ''
desc = '' 
feature_img = '' 
meta = ''
recommended = ''
genre = ''
rating = ''
maturity = ''
country = ''
try:
    title = show_soup.select('h1.e14injhv7')[0].text
    desc = show_soup.select('p[itemprop=description]')[0].text 
    feature_img = show_soup.select('.e1x40mdt0 picture.e1181ybh0 img.e1181ybh1')[0]['src']  
    recommended = show_soup.select('.e1yfir8f4 .e1qyeclq4') 
    genre = show_soup.find('a',class_='css-10wrqt0').text
    rating = show_soup.find('span', class_='ey4ir3j3').text
    maturity = show_soup.select('span[title*=rating]')[0].text
    country = show_soup.select('.css-10wrqt0[href*=origin]')[0].text
except:
    print('error')

In [ ]:
#Individual Search Dictionary
dic_list = {
    'title': title,
    'description': desc,
    'feature_img': feature_img ,
    'genre' : genre,
    'rating': rating,
    'maturity':maturity,
    'country':country,
    'recommended':[r.text for r in recommended]
} 
dic_list

In [ ]:
#Netflix Search URL
netflix_query = query.replace(" ","%20")
netfind = f"https://www.netflix.com/search?q={netflix_query}"
print(netfind)

In [ ]:
#Prime Search URL
amazon_query = query.replace(" ","+")
amazfind = f"https://www.amazon.com/s?k={amazon_query}&i=prime-instant-video&ref=nb_sb_noss_2"
print(amazfind)

In [ ]:
#YouTube Search URL
youtube_query = query.replace(" ","+")
ytubfind = f'https://www.youtube.com/results?search_query={youtube_query}'
print(ytubfind)

In [ ]:
#SERVICE PROVIDER TITLE DB
name = input('Please enter your service from this selection: (netflix, amazon, hbo, hulu)? ')
#full list: (netflix, amazon, hbo_max, hbo, showtime, hulu, fx_tveverywhere, starz, apple_tv_plus, plex_free, disney_plus, peacock, philo, fubo_tv, epix, cbs_all_access, dc_universe, mubi, discovery_plus, amc_premiere, amc, britbox, ifc, youtube_premium, shudder, criterion_channel, funimation, fandor, hoopla, kanopy, tubi, plutotv, peacock, vudu, imdb_tv, popcornflix, crunchyroll, crackle, acorntv, cinemax, hallmark_movies_now, sundance_tveverywhere, syfy_tveverywhere, tbs, tnt, bet_plus, watch_tcm, comedycentral_tveverywhere, hallmark_everywhere, lifetime_tveverywhere, disneynow, tvision)

In [ ]:
# DB FOR JUST TITLES
# for i in range(0, 50050, 50):
#     loop_url = f'https://reelgood.com/source/{name}?offset={i}'
#     html = BeautifulSoup(requests.get(f"{loop_url}").text, "html.parser")  
#     trs = html.select('table.css-1179hly tbody tr') 
#     trs
#     for tr in trs:
#         tds = tr.select('td')  
#         for td in tds:
#             movietitle_list = td.select(".e126mwsw1 a[href*='/movie/']")
#             if len(movietitle_list) > 0:
#                 moviefull = movietitle_list[0].text
#                 print(moviefull)
#             tvtitle_list = td.select(".e126mwsw1 a[href*='/show/']")
#             if len(tvtitle_list) > 0:
#                 tvfull = tvtitle_list[0].text
#                 print(tvfull)

In [ ]:
#DB FOR TITLES, YEAR, MATURITY, IMDB RATING (FIRST 50)
# box_url=f'https://reelgood.com/all'
# soup = BeautifulSoup(requests.get(f"{box_url}").text, "html.parser")  
# table = soup.find('table', class_='css-1179hly')
# tbody = table.tbody
# for tag in tbody.find_all('style'):
#     tbody.style.decompose()

# subset = pd.read_html(f'<table>{tbody}</table>')[0]
# subset.head()

In [ ]:
# #FIRST 50 CONTINUED
# cropped = movies_df[[1,3,4,5]]
# #cropped.head()
# fifty_allservice_df = cropped.rename(columns={list(cropped)[0]:'Title',list(cropped)[1]:'Year',list(cropped)[2]:'Maturity (Age)',list(cropped)[3]:'IMDB Rating'})
# fifty_allservice_df.head()

In [ ]:
#ALL TITLES FOR SERVICE FROM INPUT
for offset in range(0, 'INSERT MAX NUMBER OF RECORDS', 50):
    cycle_url = f'https://reelgood.com/source/{name}?offset={offset}'
    #print(cycle_url)
    soup = BeautifulSoup(requests.get(f"{cycle_url}").text, "html.parser")  
    frame = soup.find('table', class_='css-1179hly')
    body = frame.tbody
    for tag in body.find_all('style'):
        body.style.decompose()
    subbed = pd.read_html(f'<table>{body}</table>')[0]
    
    if offset > 0:
        combined_df = combined_df.append(subbed)
    else:
        combined_df = subbed.copy()
           
combined_df

In [ ]:
#Polish
polished = combined_df[[1,3,4,5]]
#polished.head()
allservice_df = polished.rename(columns={list(polished)[0]:'Title',list(polished)[1]:'Year',list(polished)[2]:'Maturity (Age)',list(polished)[3]:'IMDB Rating'})
allservice_df['IMDB Rating'] = allservice_df['IMDB Rating'].str.replace('/', ' out of ')
#allservice_df

In [ ]:
#TO CSV
#fifty_allservice_df.to_csv(f'{name}.csv',index=False)
allservice_df.to_csv(f'{name}.csv',index=False)

In [ ]:
#TO JSON
#derulo = allservice_df.to_json(orient='records')
#derulo